In [1]:
# class4_embeddings_assignment.ipynb (Syaqila Ghavaro Adista)

In [2]:
!pip install -q google-generativeai pandas scikit-learn numpy

In [3]:
import google.generativeai as genai
import pandas as pd
import numpy as np
from google.colab import userdata
from sklearn.metrics.pairwise import cosine_similarity
import time

In [4]:
# Konfigurasi Kunci API Gemini dari Colab Secrets

try:
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    if not GEMINI_API_KEY:
        raise ValueError("Kunci API 'GEMINI_API_KEY' tidak ditemukan di Colab Secrets.")
    genai.configure(api_key=GEMINI_API_KEY)
    print("Kunci API Gemini berhasil dikonfigurasi! 🎉")
except Exception as e:
    print(f"Error saat konfigurasi Kunci API: {e}")
    print("Pastikan kamu sudah mengatur 'GEMINI_API_KEY' di Colab Secrets.")
    GEMINI_API_KEY = None # Set ke None jika gagal

Kunci API Gemini berhasil dikonfigurasi! 🎉


In [5]:
# Model untuk generasi teks (membuat dataset sintetis)
# Pastikan model ini mendukung generasi teks yang cukup panjang jika diperlukan

if GEMINI_API_KEY:
    generative_model = genai.GenerativeModel(
        model_name='gemini-1.5-flash-latest' # Atau model lain yang cocok untuk generasi
    )
    print(f"Model Generatif ({generative_model.model_name}) berhasil diinisialisasi.")

    # Model untuk membuat embeddings
    # model_name='text-embedding-004' adalah salah satu model embedding dari Google
    embedding_model_name = 'text-embedding-004'
    print(f"Akan menggunakan model Embedding: {embedding_model_name}")
else:
    print("Model tidak dapat diinisialisasi karena Kunci API tidak tersedia.")
    generative_model = None
    embedding_model_name = None

Model Generatif (models/gemini-1.5-flash-latest) berhasil diinisialisasi.
Akan menggunakan model Embedding: text-embedding-004


In [6]:
def generate_synthetic_data(label, aspek_spesifik, jumlah_sampel=5):
    """Membangkitkan data sintetis untuk label tertentu menggunakan LLM."""
    if not generative_model:
        print("Model generatif tidak tersedia.")
        return []

    prompt = f"""Tolong buatkan {jumlah_sampel} contoh ulasan singkat (antara 20 sampai 50 kata) untuk produk fiksi dalam kategori '{label}'.
Fokus pada {aspek_spesifik}.
Pastikan ulasan terdengar alami dan bervariasi (ada yang positif, netral, atau mungkin sedikit kritis).
Setiap ulasan harus diawali dengan "Ulasan X: " di mana X adalah nomor ulasan.
Pisahkan setiap ulasan dengan baris baru.

Contoh Format:
Ulasan 1: Teks ulasan pertama.
Ulasan 2: Teks ulasan kedua.
"""
    print(f"Mengirim prompt untuk label: {label}...")
    try:
        response = generative_model.generate_content(prompt)
        # Membersihkan dan memisahkan teks ulasan
        raw_text = response.text
        # Mencari pola "Ulasan X: " dan mengambil teks setelahnya
        reviews = []
        for line in raw_text.split('\n'):
            if line.startswith("Ulasan ") and ": " in line:
                review_text = line.split(": ", 1)[1]
                if review_text: # Pastikan ada teks setelah ": "
                    reviews.append(review_text.strip())
        return reviews[:jumlah_sampel] # Pastikan jumlahnya sesuai permintaan
    except Exception as e:
        print(f"Gagal membangkitkan data untuk {label}: {e}")
        return []

# Definisikan label dan aspek spesifiknya
labels_and_aspects = {
    'smartphone_gaming': 'performa grafis tinggi dan daya tahan baterai saat bermain game',
    'laptop_ultralight': 'portabilitas, desain tipis, dan kenyamanan keyboard untuk kerja mobile',
    'headphone_noise_cancelling': 'kualitas peredam bising, kenyamanan penggunaan jangka lama, dan kejernihan suara',
    'smartwatch_kesehatan': 'akurasi sensor detak jantung, pelacakan tidur, dan fitur olahraga',
    'kamera_vlogging': 'kualitas video 4K, stabilisasi gambar, dan kemudahan penggunaan untuk pemula'
}

all_texts = []
all_labels = []
samples_per_label_target = 7 # Untuk demo, ganti ke 20-50 untuk tugas sebenarnya

if GEMINI_API_KEY:
    for label, aspek in labels_and_aspects.items():
        print(f"\nGenerating data untuk: {label}")
        generated_reviews = generate_synthetic_data(label, aspek, samples_per_label_target)
        if generated_reviews:
            all_texts.extend(generated_reviews)
            all_labels.extend([label] * len(generated_reviews))
            print(f"Berhasil generate {len(generated_reviews)} ulasan untuk {label}.")
        else:
            print(f"Tidak ada ulasan yang di-generate atau ada error untuk {label}.")
        time.sleep(2) # Jeda antar panggilan API untuk menghindari rate limit jika ada
else:
    print("Tidak dapat membangkitkan data karena Kunci API tidak ada.")


Generating data untuk: smartphone_gaming
Mengirim prompt untuk label: smartphone_gaming...
Berhasil generate 7 ulasan untuk smartphone_gaming.

Generating data untuk: laptop_ultralight
Mengirim prompt untuk label: laptop_ultralight...
Berhasil generate 7 ulasan untuk laptop_ultralight.

Generating data untuk: headphone_noise_cancelling
Mengirim prompt untuk label: headphone_noise_cancelling...
Berhasil generate 7 ulasan untuk headphone_noise_cancelling.

Generating data untuk: smartwatch_kesehatan
Mengirim prompt untuk label: smartwatch_kesehatan...
Berhasil generate 7 ulasan untuk smartwatch_kesehatan.

Generating data untuk: kamera_vlogging
Mengirim prompt untuk label: kamera_vlogging...
Berhasil generate 7 ulasan untuk kamera_vlogging.


In [7]:
if all_texts and all_labels:
    df_synthetic = pd.DataFrame({
        'text': all_texts,
        'label': all_labels
    })
    print(f"\nTotal data berhasil dibuat: {len(df_synthetic)} sampel.")
    print("Contoh 5 data pertama:")
    display(df_synthetic.head())
    print("\nDistribusi label:")
    print(df_synthetic['label'].value_counts())
else:
    print("Tidak ada data sintetis yang berhasil dibuat. DataFrame kosong.")
    df_synthetic = pd.DataFrame(columns=['text', 'label']) # Buat DataFrame kosong



Total data berhasil dibuat: 35 sampel.
Contoh 5 data pertama:


,text,label
0,Grafisnya luar biasa! Game berat jalan mulus ...,smartphone_gaming
1,"Performa gaming oke punya, grafis detail dan s...",smartphone_gaming
2,Smartphone gaming yang cukup baik. Grafisnya ...,smartphone_gaming
3,Wow! Grafisnya realistis banget! Baterai jug...,smartphone_gaming
4,"Untuk harga segitu, performa grafisnya cukup m...",smartphone_gaming



Distribusi label:
label
smartphone_gaming             7
laptop_ultralight             7
headphone_noise_cancelling    7
smartwatch_kesehatan          7
kamera_vlogging               7
Name: count, dtype: int64


In [8]:
def get_embeddings(texts, model_name):
    """Mendapatkan embeddings untuk daftar teks."""
    if not GEMINI_API_KEY or not model_name:
        print("Kunci API atau nama model embedding tidak tersedia.")
        return [None] * len(texts)
    try:
        # API genai.embed_content bisa menerima list of texts
        # dan secara otomatis melakukan batching jika diperlukan (tergantung implementasi API).
        # Kita akan mengirimkan list teks langsung.
        # Task_type bisa 'RETRIEVAL_DOCUMENT' untuk dokumen yang akan dicari,
        # atau 'SEMANTIC_SIMILARITY' untuk tugas kesamaan umum.
        result = genai.embed_content(
            model=f"models/{model_name}",
            content=texts,
            task_type="RETRIEVAL_DOCUMENT" # atau "SEMANTIC_SIMILARITY" / "RETRIEVAL_QUERY" dll.
        )
        return [np.array(embedding) for embedding in result['embedding']]
    except Exception as e:
        print(f"Error saat membuat embeddings: {e}")
        return [None] * len(texts) # Kembalikan list of None jika error

if not df_synthetic.empty and embedding_model_name:
    print(f"Membuat embeddings untuk {len(df_synthetic)} teks menggunakan {embedding_model_name}...")
    # Karena jumlah teks kita sedikit, kita bisa proses sekaligus.
    # Untuk dataset besar, pertimbangkan batching manual jika API tidak menangani secara optimal.
    df_synthetic['embedding'] = get_embeddings(df_synthetic['text'].tolist(), embedding_model_name)

    # Cek apakah ada embedding yang gagal (None)
    failed_embeddings = df_synthetic['embedding'].isnull().sum()
    if failed_embeddings > 0:
        print(f"PERINGATAN: Terdapat {failed_embeddings} teks yang gagal di-embed.")
        # Hapus baris yang embeddingnya gagal
        df_synthetic.dropna(subset=['embedding'], inplace=True)
        df_synthetic.reset_index(drop=True, inplace=True)
        print(f"Jumlah data setelah menghapus yang gagal: {len(df_synthetic)}")

    if not df_synthetic.empty:
        print("\nBerhasil membuat embeddings. Contoh embedding pertama:")
        print(df_synthetic['embedding'].iloc[0][:10]) # Tampilkan 10 elemen pertama dari embedding pertama
        print(f"Dimensi embedding: {len(df_synthetic['embedding'].iloc[0])}")
        display(df_synthetic.head())
    else:
        print("Tidak ada data untuk di-embed setelah penghapusan gagal.")
else:
    print("DataFrame sintetis kosong atau model embedding tidak diset, tidak bisa membuat embeddings.")


Membuat embeddings untuk 35 teks menggunakan text-embedding-004...

Berhasil membuat embeddings. Contoh embedding pertama:
[-0.03042268  0.00178165 -0.02654048 -0.03883928  0.02102823  0.00427682
  0.00445984  0.01432995 -0.00649296 -0.00146519]
Dimensi embedding: 768


,text,label,embedding
0,Grafisnya luar biasa! Game berat jalan mulus ...,smartphone_gaming,"[-0.03042268, 0.001781645, -0.02654048, -0.038..."
1,"Performa gaming oke punya, grafis detail dan s...",smartphone_gaming,"[-0.017029691, 0.009143128, -0.047730576, -0.0..."
2,Smartphone gaming yang cukup baik. Grafisnya ...,smartphone_gaming,"[-0.029588822, 0.017507408, -0.05034199, -0.05..."
3,Wow! Grafisnya realistis banget! Baterai jug...,smartphone_gaming,"[-0.02580021, 0.0016736267, -0.04040996, -0.04..."
4,"Untuk harga segitu, performa grafisnya cukup m...",smartphone_gaming,"[-0.004647071, 0.0014684419, -0.061317988, -0...."


In [9]:
def semantic_search(query_text, df_data, embedding_model, top_n=3):
    """
    Melakukan pencarian semantik.
    Args:
        query_text (str): Teks query dari pengguna.
        df_data (pd.DataFrame): DataFrame yang berisi kolom 'text' dan 'embedding'.
        embedding_model (str): Nama model embedding yang digunakan.
        top_n (int): Jumlah hasil teratas yang ingin ditampilkan.
    Returns:
        pd.DataFrame: DataFrame berisi hasil pencarian teratas.
    """
    if df_data.empty or 'embedding' not in df_data.columns:
        print("DataFrame kosong atau tidak ada kolom 'embedding'.")
        return pd.DataFrame()
    if not GEMINI_API_KEY or not embedding_model:
        print("Kunci API atau nama model embedding tidak tersedia untuk query.")
        return pd.DataFrame()

    print(f"\n🔍 Mencari untuk query: '{query_text}'")

    # 1. Buat embedding untuk query
    try:
        query_embedding_result = genai.embed_content(
            model=f"models/{embedding_model}",
            content=query_text,
            task_type="RETRIEVAL_QUERY" # Gunakan task_type yang sesuai untuk query
        )
        query_embedding = np.array(query_embedding_result['embedding'])
    except Exception as e:
        print(f"Error saat membuat embedding untuk query: {e}")
        return pd.DataFrame()

    if query_embedding is None:
        print("Gagal membuat embedding untuk query.")
        return pd.DataFrame()

    # 2. Hitung cosine similarity antara query embedding dan semua text embedding di dataset
    # Pastikan semua embeddings adalah numpy array dan memiliki dimensi yang sama
    corpus_embeddings = np.array(df_data['embedding'].tolist())
    similarities = cosine_similarity(query_embedding.reshape(1, -1), corpus_embeddings)

    # 3. Dapatkan skor similarity dan urutkan
    # similarities[0] karena query_embedding direshape jadi (1, dim)
    df_data['similarity_score'] = similarities[0]
    results_df = df_data.sort_values(by='similarity_score', ascending=False).head(top_n)

    return results_df[['text', 'label', 'similarity_score']]

In [10]:
if not df_synthetic.empty and 'embedding' in df_synthetic.columns and embedding_model_name:
    # Contoh query 1
    query1 = "laptop tipis buat kerja dari cafe"
    hasil_pencarian1 = semantic_search(query1, df_synthetic.copy(), embedding_model_name, top_n=3)
    if not hasil_pencarian1.empty:
        print(f"Hasil pencarian untuk query: '{query1}'")
        display(hasil_pencarian1)
    else:
        print(f"Tidak ada hasil untuk query: '{query1}'")

    # Contoh query 2
    query2 = "main game berat grafisnya mantap"
    hasil_pencarian2 = semantic_search(query2, df_synthetic.copy(), embedding_model_name, top_n=3)
    if not hasil_pencarian2.empty:
        print(f"\nHasil pencarian untuk query: '{query2}'")
        display(hasil_pencarian2)
    else:
        print(f"Tidak ada hasil untuk query: '{query2}'")

    # Contoh query 3
    query3 = "headphone yang bisa bikin suara luar nggak kedengeran"
    hasil_pencarian3 = semantic_search(query3, df_synthetic.copy(), embedding_model_name, top_n=3)
    if not hasil_pencarian3.empty:
        print(f"\nHasil pencarian untuk query: '{query3}'")
        display(hasil_pencarian3)
    else:
        print(f"Tidak ada hasil untuk query: '{query3}'")
else:
    print("Tidak bisa melakukan pencarian semantik karena dataset atau embeddings tidak siap.")



🔍 Mencari untuk query: 'laptop tipis buat kerja dari cafe'
Hasil pencarian untuk query: 'laptop tipis buat kerja dari cafe'


,text,label,similarity_score
12,Portabilitasnya sempurna untuk saya yang serin...,laptop_ultralight,0.642342
7,"Laptop ultralight yang luar biasa! Tipis, rin...",laptop_ultralight,0.641510
10,Laptop ini benar-benar tipis dan ringan! Sang...,laptop_ultralight,0.631412



🔍 Mencari untuk query: 'main game berat grafisnya mantap'

Hasil pencarian untuk query: 'main game berat grafisnya mantap'


,text,label,similarity_score
0,Grafisnya luar biasa! Game berat jalan mulus ...,smartphone_gaming,0.667278
5,"Baterai lumayan tahan lama, bahkan saat main g...",smartphone_gaming,0.615951
1,"Performa gaming oke punya, grafis detail dan s...",smartphone_gaming,0.609013



🔍 Mencari untuk query: 'headphone yang bisa bikin suara luar nggak kedengeran'

Hasil pencarian untuk query: 'headphone yang bisa bikin suara luar nggak kedengeran'


,text,label,similarity_score
20,Headphone yang nyaman untuk penggunaan jangka ...,headphone_noise_cancelling,0.692796
15,"Headphone ini cukup nyaman, peredam bisingnya...",headphone_noise_cancelling,0.687984
19,Peredam bising yang sangat membantu saat beker...,headphone_noise_cancelling,0.680347
